In [59]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

In [189]:
# Define the ticker symbols as a list
ticker_symbols = ["AMZN", "MSFT"]
window_days = 90
# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)

# Create an empty DataFrame to store the adjusted data
adjusted_stock_data = pd.DataFrame()
earnings_data = {}  # Dictionary to store earnings dates for each ticker
metric_list = []

# Fetch historical data for each ticker symbol and adjust the format
for symbol in ticker_symbols:

    # symbol's historical price related
    data = yf.download(symbol, start=start_date, end=end_date)
    data = data.sort_index(ascending=False)
    data.index = [date.date() for date in data.index.sort_values(ascending=False)] # convert original timestamp index to date index

    # symbol's earning related
    stock = yf.Ticker(symbol)
    earnings_dates = stock.get_earnings_dates(limit=12)  # Fetch earnings dates for the past 4 quarters and forecast for the next 4 quarters
    earnings_data[symbol] = earnings_dates


    # Extract close prices and add them to the adjusted DataFrame with ticker symbol as column name
    adjusted_stock_data[symbol] = data['Close'].round(2)
    adjusted_stock_data[symbol + "_EPS_TTM"] = None


    # EPS TTM calculation
    EPS_TTM_start_date = data.index
    for i in EPS_TTM_start_date:
        for key, value in earnings_data.items():
            if key == symbol:
                symbol_hist_EPS = value['Reported EPS'][value['Reported EPS'].notnull()]
                symbol_hist_EPS.index = [date.date() for date in symbol_hist_EPS.index.sort_values(ascending=False)] # convert timestamp index to a date index(sorted, descending)

                # Filter the DataFrame to include only dates less than or equal to the target date
                filtered_symbol_hist_EPS = symbol_hist_EPS[symbol_hist_EPS.index <= i]
                
                # Select the first four rows from the sorted DataFrame
                closest_dates = filtered_symbol_hist_EPS.head(4)      

                # # Calculate the sum of the numeric values in the selected rows
                EPS_TTM = closest_dates.sum()

                adjusted_stock_data.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM

            else:
                continue
            

    # stock's stats

    adjusted_stock_data[f"{symbol}_PE_TTM"] = adjusted_stock_data[symbol] / adjusted_stock_data[f"{symbol}_EPS_TTM"]
    adjusted_stock_data[f"{symbol}_PE_TTM_avg"] = adjusted_stock_data[f"{symbol}_PE_TTM"].mean().round(2)
    adjusted_stock_data[f"{symbol}_PE_TTM_std"] = np.std(adjusted_stock_data[f"{symbol}_PE_TTM"]).round(2)
    adjusted_stock_data[f"{symbol}_PE_TTM_volatility_+"] = adjusted_stock_data[f"{symbol}_PE_TTM_avg"] + adjusted_stock_data[f"{symbol}_PE_TTM_std"] # 这个是PE的波动范围上限
    adjusted_stock_data[f"{symbol}_PE_TTM_volatility_-"] = adjusted_stock_data[f"{symbol}_PE_TTM_avg"] - adjusted_stock_data[f"{symbol}_PE_TTM_std"] # 这个是PE的波动范围下限

    adjusted_stock_data[f"{symbol}_relative_valuation_+"] = adjusted_stock_data[f"{symbol}_PE_TTM_volatility_+"] * adjusted_stock_data[f"{symbol}_PE_TTM"] # 这个是relative valuation的价格上限
    adjusted_stock_data[f"{symbol}_relative_valuation_-"] = adjusted_stock_data[f"{symbol}_PE_TTM_volatility_-"] * adjusted_stock_data[f"{symbol}_PE_TTM"] # 这个是relative valuation的价格下限

    adjusted_stock_data[f"{symbol}_price_min"] = adjusted_stock_data[symbol].min().round(2)
    adjusted_stock_data[f"{symbol}_price_max"] = adjusted_stock_data[symbol].max().round(2)
    adjusted_stock_data[f"{symbol}_price_avg"] = adjusted_stock_data[symbol].mean().round(2)
    adjusted_stock_data[f"{symbol}_{window_days}_price_std"] = np.std(adjusted_stock_data[symbol]).round(2)

# # Reset index to make it cleaner
# adjusted_stock_data.reset_index(inplace=True)

# # Example usage: Display the first few rows of the adjusted DataFrame
# adjusted_stock_data.sort_values(by='Date', ascending=False)

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

adjusted_stock_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AMZN,AMZN_EPS_TTM,AMZN_PE_TTM,AMZN_PE_TTM_std,AMZN_price_min,AMZN_price_max,AMZN_price_avg,AMZN_90_price_std,MSFT,MSFT_EPS_TTM,MSFT_PE_TTM,MSFT_PE_TTM_std,MSFT_price_min,MSFT_price_max,MSFT_price_avg,MSFT_90_price_std
2024-04-12,186.13,2.9,64.182759,5.07,151.71,189.05,172.12,9.43,421.90,11.06,38.146474,0.78,389.47,429.37,412.01,9.91
2024-04-11,189.05,2.9,65.189655,5.07,151.71,189.05,172.12,9.43,427.93,11.06,38.691682,0.78,389.47,429.37,412.01,9.91
2024-04-10,185.95,2.9,64.12069,5.07,151.71,189.05,172.12,9.43,423.26,11.06,38.269439,0.78,389.47,429.37,412.01,9.91
2024-04-09,185.67,2.9,64.024138,5.07,151.71,189.05,172.12,9.43,426.28,11.06,38.542495,0.78,389.47,429.37,412.01,9.91
2024-04-08,185.19,2.9,63.858621,5.07,151.71,189.05,172.12,9.43,424.59,11.06,38.389693,0.78,389.47,429.37,412.01,9.91
2024-04-05,185.07,2.9,63.817241,5.07,151.71,189.05,172.12,9.43,425.52,11.06,38.473779,0.78,389.47,429.37,412.01,9.91
2024-04-04,180.00,2.9,62.068966,5.07,151.71,189.05,172.12,9.43,417.88,11.06,37.783002,0.78,389.47,429.37,412.01,9.91
2024-04-03,182.41,2.9,62.9,5.07,151.71,189.05,172.12,9.43,420.45,11.06,38.015371,0.78,389.47,429.37,412.01,9.91
2024-04-02,180.69,2.9,62.306897,5.07,151.71,189.05,172.12,9.43,421.44,11.06,38.104882,0.78,389.47,429.37,412.01,9.91
2024-04-01,180.97,2.9,62.403448,5.07,151.71,189.05,172.12,9.43,424.57,11.06,38.387884,0.78,389.47,429.37,412.01,9.91


In [138]:
data.index.sort_values(ascending=False)

DatetimeIndex(['2024-04-12', '2024-04-11', '2024-04-10', '2024-04-09',
               '2024-04-08', '2024-04-05', '2024-04-04', '2024-04-03',
               '2024-04-02', '2024-04-01', '2024-03-28', '2024-03-27',
               '2024-03-26', '2024-03-25', '2024-03-22', '2024-03-21',
               '2024-03-20', '2024-03-19', '2024-03-18', '2024-03-15',
               '2024-03-14', '2024-03-13', '2024-03-12', '2024-03-11',
               '2024-03-08', '2024-03-07', '2024-03-06', '2024-03-05',
               '2024-03-04', '2024-03-01', '2024-02-29', '2024-02-28',
               '2024-02-27', '2024-02-26', '2024-02-23', '2024-02-22',
               '2024-02-21', '2024-02-20', '2024-02-16', '2024-02-15',
               '2024-02-14', '2024-02-13', '2024-02-12', '2024-02-09',
               '2024-02-08', '2024-02-07', '2024-02-06', '2024-02-05',
               '2024-02-02', '2024-02-01', '2024-01-31', '2024-01-30',
               '2024-01-29', '2024-01-26', '2024-01-25', '2024-01-24',
      

In [117]:
for value in earnings_data.values():
    print((value['Reported EPS'][value['Reported EPS'].notnull()]).index)

DatetimeIndex(['2024-01-30 16:00:00-05:00', '2023-10-24 16:00:00-04:00',
               '2023-07-25 16:00:00-04:00', '2023-04-25 16:00:00-04:00',
               '2023-02-02 16:00:00-05:00', '2022-10-25 16:00:00-04:00',
               '2022-07-26 16:00:00-04:00', '2022-04-26 16:00:00-04:00'],
              dtype='datetime64[ns, America/New_York]', name='Earnings Date', freq=None)


In [66]:
EPS_TTM_start_date, EPS_TTM_end_date

(DatetimeIndex(['2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25',
                '2024-01-26', '2024-01-29', '2024-01-30', '2024-01-31',
                '2024-02-01', '2024-02-02', '2024-02-05', '2024-02-06',
                '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-12',
                '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16',
                '2024-02-20', '2024-02-21', '2024-02-22', '2024-02-23',
                '2024-02-26', '2024-02-27', '2024-02-28', '2024-02-29',
                '2024-03-01', '2024-03-04', '2024-03-05', '2024-03-06',
                '2024-03-07', '2024-03-08', '2024-03-11', '2024-03-12',
                '2024-03-13', '2024-03-14', '2024-03-15', '2024-03-18',
                '2024-03-19', '2024-03-20', '2024-03-21', '2024-03-22',
                '2024-03-25', '2024-03-26', '2024-03-27', '2024-03-28',
                '2024-04-01', '2024-04-02', '2024-04-03', '2024-

In [104]:
# Assuming df is your DataFrame with a timestamp index
# Example DataFrame creation
dates = pd.date_range(start='2022-01-01', end='2022-01-05', freq='D')
data = {'values': [1, 2, 3, 4, 5]}
df = pd.DataFrame(data, index=dates)

In [105]:
df

,values
2022-01-01,1
2022-01-02,2
2022-01-03,3
2022-01-04,4
2022-01-05,5


In [67]:
earnings_data

{'GOOGL':                            EPS Estimate  Reported EPS  Surprise(%)
 Earnings Date                                                     
 2025-01-28 16:00:00-05:00           NaN           NaN          NaN
 2024-10-22 16:00:00-04:00           NaN           NaN          NaN
 2024-07-23 16:00:00-04:00           NaN           NaN          NaN
 2024-04-23 16:00:00-04:00          1.51           NaN          NaN
 2024-01-30 16:00:00-05:00          1.59          1.64       0.0298
 2023-10-24 16:00:00-04:00          1.45          1.55       0.0684
 2023-07-25 16:00:00-04:00          1.34          1.44       0.0754
 2023-04-25 16:00:00-04:00          1.07          1.17       0.0972
 2023-02-02 16:00:00-05:00          1.18          1.05      -0.1073
 2022-10-25 16:00:00-04:00          1.25          1.06      -0.1514
 2022-07-26 16:00:00-04:00          1.28          1.21      -0.0541
 2022-04-26 16:00:00-04:00          1.30          1.23      -0.0500,
 'AAPL':                            EP

In [33]:
def get_earnings_dates(ticker_symbols):
    earnings_data = {}  # Dictionary to store earnings dates for each ticker
    
    for ticker in ticker_symbols:
        stock = yf.Ticker(ticker)
        earnings_dates = stock.get_earnings_dates(limit=12)  # Fetch earnings dates for the past 4 quarters and forecast for the next 4 quarters
        earnings_data[ticker] = earnings_dates
        
    return earnings_data


earnings_data = get_earnings_dates(ticker_symbols)


for ticker, dates in earnings_data.items():
    if ticker in ['AAPL','AMZN']:
        print(f"Earnings Dates for {ticker}:")
        print(dates)
        print()
    else:
        continue

Earnings Dates for AAPL:
                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-01-30 16:00:00-05:00           NaN           NaN          NaN
2024-10-31 06:00:00-04:00           NaN           NaN          NaN
2024-08-01 06:00:00-04:00           NaN           NaN          NaN
2024-05-02 06:00:00-04:00          1.51           NaN          NaN
2024-02-01 16:00:00-05:00          2.10          2.18       0.0390
2023-11-02 16:00:00-04:00          1.39          1.46       0.0492
2023-08-03 16:00:00-04:00          1.19          1.26       0.0549
2023-05-04 16:00:00-04:00          1.43          1.52       0.0603
2023-02-02 16:00:00-05:00          1.94          1.88      -0.0288
2022-10-27 16:00:00-04:00          1.27          1.29       0.0155
2022-07-28 16:00:00-04:00          1.16          1.20       0.0325
2022-04-28 16:00:00-04:00          1.43          1.52       0.0644

Earnings Dates for AMZN:
           

In [36]:
earnings_data = get_earnings_dates(ticker_symbols)


for ticker, dates in earnings_data.items():
    if ticker == 'AAPL':
        print(f"Earnings Dates for {ticker}:")
        print(dates)
        print()

Earnings Dates for AAPL:
                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-01-30 16:00:00-05:00           NaN           NaN          NaN
2024-10-31 06:00:00-04:00           NaN           NaN          NaN
2024-08-01 06:00:00-04:00           NaN           NaN          NaN
2024-05-02 06:00:00-04:00          1.51           NaN          NaN
2024-02-01 16:00:00-05:00          2.10          2.18       0.0390
2023-11-02 16:00:00-04:00          1.39          1.46       0.0492
2023-08-03 16:00:00-04:00          1.19          1.26       0.0549
2023-05-04 16:00:00-04:00          1.43          1.52       0.0603
2023-02-02 16:00:00-05:00          1.94          1.88      -0.0288
2022-10-27 16:00:00-04:00          1.27          1.29       0.0155
2022-07-28 16:00:00-04:00          1.16          1.20       0.0325
2022-04-28 16:00:00-04:00          1.43          1.52       0.0644



In [41]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

def get_earnings_dates(ticker_symbols):
    earnings_data = {}  # Dictionary to store earnings dates for each ticker
    
    for ticker in ticker_symbols:
        stock = yf.Ticker(ticker)
        earnings_dates = stock.get_earnings_dates(limit=12)  # Fetch earnings dates for the past 4 quarters and forecast for the next 4 quarters
        earnings_data[ticker] = earnings_dates
        
    return earnings_data

def calculate_ttm_pe(stock_data, earnings_data):
    # Create a DataFrame to store TTM P/E ratio
    ttm_pe_df = stock_data.copy()
    ttm_pe_df['TTM_PE'] = None
    
    for ticker, earnings in earnings_data.items():  # Changed 'earnings_dates' to 'earnings'
        
        for index, row in ttm_pe_df.iterrows():
            current_date = datetime.strptime(row['Date'], '%Y-%m-%d')  # Convert date string to datetime object
            ttm_earnings = 0
            
            # Calculate TTM earnings
            for earnings_date, earning in earnings.iterrows():
                earnings_date = datetime.strptime(str(earnings_date), '%Y-%m-%d')  # Convert earnings date to string before parsing
                if earnings_date <= current_date and earnings_date >= current_date - timedelta(days=365):
                    ttm_earnings += earning['EPS']
                    
            # Calculate TTM P/E ratio
            if ttm_earnings != 0:
                ttm_pe = row[ticker] / ttm_earnings
                ttm_pe_df.at[index, 'TTM_PE'] = ttm_pe
                
    return ttm_pe_df

# Sample stock data
stock_data = pd.DataFrame({
    'Date': ['2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-22',
             '2024-04-08', '2024-04-09', '2024-04-10', '2024-04-11', '2024-04-12'],
    'AAPL': [183.63, 182.68, 188.63, 191.56, 193.89, 168.45, 169.67, 167.78, 175.04, 176.55]
})

# Sample earnings data (replace with your actual earnings data)
stock_earnings = {
    'AAPL': pd.DataFrame({
        'Date': ['2023-01-16', '2023-04-16', '2023-07-16', '2023-10-16', '2024-01-16'],
        'EPS': [3.28, 2.55, 2.33, 2.49, 2.91]
    })
}

# Example usage
earnings_data = {'AAPL': stock_earnings['AAPL']}  # Replace with your actual earnings data
ttm_pe_df = calculate_ttm_pe(stock_data, earnings_data)
print(ttm_pe_df)


ValueError: time data '0' does not match format '%Y-%m-%d'

In [12]:
# Calculate the standard deviation
std_deviation = np.std(close_price)

# Print the standard deviation
print("Standard Deviation of Closing Prices for the past 90 days:", std_deviation)

Standard Deviation of Closing Prices for the past 90 days: 6.652266721389418


In [38]:
pip install jupyterlab-linenumbers

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement jupyterlab-linenumbers (from versions: none)
ERROR: No matching distribution found for jupyterlab-linenumbers

[notice] A new release of pip is available: 23.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
